In [1]:
import numpy as np
from astropy.table import Table

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

POPSYCLE SIM

In [2]:
pops = Table.read("data/research_paper/popsycle_sim.fits").to_pandas()

In [18]:
post_EW = Table.read("data/research_paper/OB110462_EW_post.fits").to_pandas() #no color data
ob = Table.read("data/research_paper/OB110462_DW_post.fits").to_pandas() #color data

In [19]:
arr = []
for i in range(len(pops)):
    if (pops["rem_id_L"][i] == 103 and (pops["mass_L"][i] > 2.5 and pops["mass_L"][i] < 6)): #.5 for added padding to mass gap
        arr.append(i) #lower mass gap of bhs is between 2.5 or so and 5-7

In [ ]:
"""For each microlensing lensing event, 9 columns correspond to properties 
of the lens (_L suffix), and 9 columns to the source (_S suffix):
* Object type where 0 = star, 101 = white dwarf, 
  102 = neutron star, 103 = black hole (rem_id);
* Mass of the object in solar masses (mass);
* Galactic radial distance in kpc (rad); 
* Galactic latitude in deg (glat); 
* Galactic longitude in deg (glon);
* Galactic radial velocity in km/s (vr);
* Galactic proper motion vector long/lat components in mas/yr (mu_b, mu_lcosb); 
* I-band magnitude (Imag).

8 columns correspond to properties of the lensing event, 
so they do not have a _L or _S suffix:
* Einstein radius in mas (theta_E);
* impact parameter (u0);
* source-lens relative proper motion in mas/yr (mu_rel);
* time of closest approach in days (t0);
* Einstein crossing time in days (t_E);
* lens-source relative parallax in mas (pi_rel);
* microlensing parallax (pi_E);
* source flux fraction in I-band (bsff_I)."""

OB110462 - Mass gap classified data

In [ ]:
"""There are 19 columns:

* time of closest approach in MJD (t0); ####################
* impact parameter (u0_amp); ####################
* Einstein crossing time in days (tE); ####################
* log10 of Einstein radius in mas (log10_thetaE); ####################

* microlensing parallax vector (piE_E, piE_N); ####################
* source position vector at t0 in arcsec (xS0_E, xS0_N);
* source proper motion vector in mas/yr (muS_E, muS_N); ####################
* lens mass in solar masses (mL); ####################

* lens-source relative parallax in mas (piRel); ####################
* lens proper motion vector in mas/yr (muL_E, muL_N);####################
* source-lens relative proper motion vector in mas/yr (muRel_E, muRel_N).####################

For each microlensing lensing event, 9 columns correspond to properties 
of the lens (_L suffix), and 9 columns to the source (_S suffix):
* Object type where 0 = star, 101 = white dwarf, 
  102 = neutron star, 103 = black hole (rem_id);
* Mass of the object in solar masses (mass); ####################
* Galactic proper motion vector long/lat components in mas/yr (mu_b, mu_lcosb); ####################

8 columns correspond to properties of the lensing event, 
so they do not have a _L or _S suffix:

* Einstein radius in mas (theta_E); ####################
* impact parameter (u0); ####################
* source-lens relative proper motion in mas/yr (mu_rel); ####################
* time of closest approach in days (t0); ####################
* Einstein crossing time in days (t_E); ####################
* lens-source relative parallax in mas (pi_rel); ####################
* microlensing parallax (pi_E);####################



"""

In [20]:
popsCols = list(pops.columns)
obCols = list(ob.columns)

In [21]:
len(popsCols)

26

Corresponding features from popsycle to ligo following:

1. 